In [22]:
import warnings
import random
warnings.filterwarnings("ignore")

import os
import pathlib
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

# For image 
import pickle
import cv2
import skimage
from skimage.util import random_noise #https://scikit-image.org/docs/dev/api/skimage.util.html

In [23]:
class SIDD_Preprocessing():

  def __init__(self,dataset_path):
    self.dataset_path = dataset_path
    self.image_size = 128
    self.size = (self.image_size,self.image_size)
    self.noisy_img_array = []
    self.gt_img_array = []

    




  def read_images(self):
    
    path = self.dataset_path + '_ReadMe.txt'
    with open(path) as f:
        readme = f.read()
    
    print(readme)









  def split_paths(self):

    noisy_images_path = []
    gt_images_path = []

    path = self.dataset_path + 'Scene_Instances.txt'
    with open(path) as f:
        instances = f.read()


    instances = instances.split('\n')
    diff_path = self.dataset_path + 'Data' + '/' 


    for f in instances:
        images_path = diff_path + f + '/'
    
        for g in os.listdir(images_path):
          image_path = images_path + g
        
          if 'NOISY' in image_path:
              noisy_images_path.append(image_path)
          else:
              gt_images_path.append(image_path)
            

    print('TOTAL NOISY IMAGES:', len(noisy_images_path))
    print('TOTAL GROUND TRUTH IMAGES:', len(gt_images_path))

    return noisy_images_path , gt_images_path






  
  def img_transformation(self, noisy_images_path, gt_images_path):



    for f in tqdm(noisy_images_path):
        img = cv2.imread(f)
        img = cv2.resize(img, self.size)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        norm = np.zeros((self.size))
        norm_image = cv2.normalize(img,  norm, 0, 255, cv2.NORM_MINMAX)
        self.noisy_img_array.append(norm_image)
    
    

    for f in tqdm(gt_images_path):
        img = cv2.imread(f)
        img = cv2.resize(img, self.size)

    
    
        self.gt_img_array.append(img)

    return self.noisy_img_array , self.gt_img_array




  def view_image(self, num = 10):

     for i in range(0,len(self.noisy_img_array),num):
    
        fig=plt.figure(figsize=(50, 25))
    
        ax = plt.subplot(131)
        plt.title('Ground Truth')
        plt.imshow(self.gt_img_array[i])
    
        ax = plt.subplot(132)
        plt.title('Noisy Image')
        plt.imshow(self.noisy_img_array[i])
        plt.show()





  def train_test_split(self,):

    noisy_train_set , noisy_test_set , gt_train_set , gt_test_set =  train_test_split(
                                                                         self.noisy_img_array , self.gt_img_array, test_size=0.20, random_state=42)

    noisy_train_set_tensor = tf.convert_to_tensor(noisy_train_set)
    noisy_test_set_tensor = tf.convert_to_tensor(noisy_test_set)
    gt_train_set_tensor = tf.convert_to_tensor(gt_train_set)
    gt_test_set_tensor = tf.convert_to_tensor(gt_test_set)


    print(noisy_train_set_tensor.shape)
    print(noisy_test_set_tensor.shape)
    print(gt_train_set_tensor.shape)
    print(gt_test_set_tensor.shape) 

    
    return noisy_train_set_tensor, noisy_test_set_tensor, gt_train_set_tensor, gt_test_set_tensor


In [ ]:
obj = SIDD_Preprocessing(dataset_path = '/content/drive/MyDrive/Colab Notebooks/SIDD_Small_sRGB_Only/')
# obj.read_images()
noisy_images_path , gt_images_path = obj.split_paths()
obj.img_transformation(noisy_images_path , gt_images_path)
noisy_train_set_tensor, noisy_test_set_tensor, gt_train_set_tensor, gt_test_set_tensor = obj.train_test_split()